In [1]:
%matplotlib inline

import numpy
import matplotlib.pyplot as plt

# Snuggly Beard

Many moons ago the feared pirate snuggly beard buried their treasure of golden coffee beans under the whiskey soaked drumlins of Clew Bay. Many years later the following directions to the treasure were discovered hidden in a crochet book:
 1. Set off east, 32 paces, from the Drunken Duck.
 2. Walk north 45 paces.
 3. Head south-west 22 paces.
 4. Go north-east 4 paces.
 5. Dig!
 
Having dug up snuggly beard's skeleton at 3 in the morning and measured her leg bones you place a Gaussian distribution over step size of 0.58 meters, with a standard deviation of 0.04 meters.

Unfortunately the Drunken Duck was burnt to the ground during a drunken brawl many years ago, and no evidence of its original location remains. Feeling a bit foolish for the unnecessary exhumation in the dead of night, you purchase a metal detector and, after a few hours, find the treasure. It is exactly 30 meters north of some bee hives.

Feeling rich, and like you really aught to justify taking a crowbar to that coffin, you decide to rebuild the Drunken Duck. Talking to a local relic, possibly a zombie, he tells you that the pub used to be 7 meters north-west of the bee hives. Given he is crunching through whiskey drenched oats at the time, you put a standard deviation of 5 meters on his evidence.

Relative to the bee hives, where should you be building? Combine all information and give the parameters of a Gaussian over the 2D offset from the bee hives.

Assume there is as much accuracy in direction as there is in distance, and make all offset distributions isotropic.

## 1. Extract Graphical Model

Define the coordinate system relative to bee hives, so they are at (0,0), as that is the output. Traditional map directions, i.e. coordinates (x, y) where increasing x is going east and increasing y is going north. All units meters.

Define four random variables:
 * DD - Where the drunken duck is.
 * N1 - Where you are after completing step 1 of the instructions.
 * N2 - Where you are after completing step 2 of the instructions.
 * N3 - Where you are after completing step 3 of the instructions.
 
The location of golden coffee beans is omitted, as we know it exactly, so there is no need to put a random variable on it.

There are two unary terms:
 * The whisky oat zombie suggests DD is at $\left(-\sqrt{\frac{7^2}{2}}, \sqrt{\frac{7^2}{2}}\right)$, with a covariance of $\begin{bmatrix}5^2 & 0\\0 & 5^2\end{bmatrix}$.
 * The last step of the instructions plus the known location of the treasure suggests N3 is at $\left(-\sqrt{\frac{(4*0.58)^2}{2}},30-\sqrt{\frac{(4*0.58)^2}{2}}\right)$, with a covariance of $\begin{bmatrix}(4*0.04)^2 & 0\\0 & (4*0.04)^2\end{bmatrix}$.

There are three pairwise terms, defining the offsets between the Drunken Duck and the steps of the treasure directions.
 * The expected offset from DD to N1 is $(32*0.58, 0)$, with a covariance of $\begin{bmatrix}(32*0.04)^2 & 0\\0 & (32*0.04)^2\end{bmatrix}$.
 * The expected offset from N1 to N2 is $(0, 45*0.58)$, with a covariance of $\begin{bmatrix}(45*0.04)^2 & 0\\0 & (45*0.04)^2\end{bmatrix}$.
 * The expected offset from N2 to N3 is $\left(-\sqrt{\frac{(22*0.58)^2}{2}},-\sqrt{\frac{(22*0.58)^2}{2}}\right)$, with a covariance of $\begin{bmatrix}(22*0.04)^2 & 0\\0 & (22*0.04)^2\end{bmatrix}$.

In [2]:
# 2. Define a Gaussian object, with the various operations needed.
# Not really necessary, as could all be placed inline, but keeps things cleaner,
# and abstracts the distribution.


class Gaussian:
    """This Gaussian object encapsulates the operations required for belief propagation;
    the maths behind this is kinda hairy, so best not to stare too long."""
    def __init__(self, mean, covar = None):
        """Called with either mean and covariance (2 parameters) or a Gaussian (1 parameter).
        Will also detect if the first parameter is an integer and create a 'no information'
        improper gaussian over that many dimensions if so."""
        # Internally use the precision/precision*mean representation (precision is the inverse
        # of the covariance, as that makes the operations simpler, and allows for improper
        # distributions...
        if covar is not None:
            self.prec = numpy.linalg.inv(covar)
            self.pmean = self.prec.dot(mean)
        
        elif isinstance(mean, int):
            self.prec = numpy.zeros((mean, mean))
            self.pmean = numpy.zeros(mean)
        
        else:
            self.prec = mean.prec.copy()
            self.pmean = mean.pmean.copy()

    
    def dims(self):
        return self.pmean.shape[0]
    
    
    def mean_covar(self):
        """Returns the mean and covariance of the Gaussian."""
        covar = numpy.linalg.inv(self.prec)
        mean = covar.dot(self.pmean)
        return mean, covar


    def rejig(self, ns):
        """Rejigs the structure of the model - you provide a list with None for
        new random variable indices, and an integer indexing a random variable
        from the original Gaussian. If there are any new RVs then the new Gaussian
        will be improper. Any variables the are not included will be marginalised
        out. Do not repeat random variables - that makes no mathematical sense."""
        
        # Work out which variables we are keeping, and throw away the rest...
        keep = numpy.array([k for k in ns if k!=None], dtype=int)
        if keep.shape[0]!=self.pmean.shape[0]:
            # Need to convert to mean/covariance...
            covar = numpy.linalg.inv(self.prec)
            mean = covar.dot(self.pmean)
            
            # Remove variables no longer required...
            mean = mean[keep]
            covar = covar[numpy.ix_(keep, keep)]
            
            # Convert back...
            self.prec = numpy.linalg.inv(covar)
            self.pmean = self.prec.dot(mean)
        
        else:
            self.prec = self.prec[numpy.ix_(keep, keep)]
            self.pmean = self.pmean[keep]
        
        # If there are any new indices add them in - just zeroed rows/columns
        # (Note that this creates an improper distribution)...
        if None in ns:
            zeros = [i for i in range(len(ns)) if ns[i]==None]
            zeros = [zeros[i] - i for i in range(len(zeros))]

            self.pmean = numpy.insert(self.pmean, zeros, 0.0, axis=0)
            self.prec = numpy.insert(self.prec, zeros, 0.0, axis=0)
            self.prec = numpy.insert(self.prec, zeros, 0.0, axis=1)
    
    
    def __mul__(lhs, rhs):
        """Multiplies the distributions, assumes they align and have the same dimensions."""
        self = Gaussian(lhs)
        self.pmean += rhs.pmean
        self.prec += rhs.prec
        return self
    
    
    def __imul__(self, rhs):
        """Inplace multiplication, i.e. *=."""
        self.pmean += rhs.pmean
        self.prec += rhs.prec
        return self
    
    
    def __str__(self):
        """When converted to string includes the mean, but not the covariance."""
        mean = numpy.linalg.pinv(self.prec).dot(self.pmean)
        return 'Gaussian([{}])'.format(','.join(['{:.2f}'.format(v) for v in mean]))



# Some test code...
a = Gaussian([3.0, -2.0], [[5.0, 0.0], [0.0, 5.0]])
print(a)

b = Gaussian(a)
b.rejig([0])
print(b)

c = Gaussian(a)
c.rejig([1])
print(c)

d = Gaussian(a)
d.rejig([1,0])
print(d)

e = Gaussian([4.0, -1.0], [[1.0, 0.0], [0.0, 1.0]])
f = a * e
print(f)

g = Gaussian(e)
g.rejig([1,None,0])
print(g)

h = Gaussian([6.0], [[3.0]])
print(h)

i = Gaussian(h)
i.rejig([None, 0, None])
print(i)

j = g * i
print(j)

Gaussian([3.00,-2.00])
Gaussian([3.00])
Gaussian([-2.00])
Gaussian([-2.00,3.00])
Gaussian([3.83,-1.17])
Gaussian([-1.00,0.00,4.00])
Gaussian([6.00])
Gaussian([0.00,6.00,0.00])
Gaussian([-1.00,6.00,4.00])


In [3]:
# 3. Defines classes for the random variables and factors (Unary and Pairwise; you could merge
# them into one generic object but I've left them seperate for simplicity).
# This code is all fairly generic as all belief propagations implementations are functionally
# identical, though can be organised (very) differently...

class RV:
    """Defines a random variable, which is really just storage for messages
    both arrviing and leaving from it."""
    
    def __init__(self, links):
        """Links is the number of links the RV has."""
        # Storage for the messages from/to each of the links...
        self.msg_in = [None] * links
        self.msg_out = [None] * links


    def send(self, link):
        """Calculates and stores the message it will send to the given link.
        Throws an error if it doesn't have the required messages."""
        msg = None
        for i, m in enumerate(self.msg_in):
            if i==link:
                continue
            
            if m==None:
                raise RuntimeError('Attempt to calculate message when RV has not received dependent messages.')
            
            if msg==None:
                msg = Gaussian(m)
            else:
                msg *= m
        
        self.msg_out[link] = msg


    def belief(self):
        """Returns the belief for this random variable, that is the marginal probability of
        the value of this random variable given all avaliable evidence."""
        ret = None
        
        for m in self.msg_in:
            if ret==None:
                ret = Gaussian(m)
            else:
                ret *= m
        
        return ret



class Unary:
    """Defines a unary factor, that is a function over a single random variable."""
    
    def __init__(self, dist, rv, rvl):
        """Constructed with the probability distribution over the random variable it is connected to,
        and the connection, as a random variable and the link of the random variable it is connected to."""
        self.dist = dist
        self.rv = rv
        self.rvl = rvl
    
    def send(self, link):
        """Makes it send it's message to the random variable it is connected to. link is provided for
        consistency with all other objects, but actually ignored."""
        self.rv.msg_in[self.rvl] = self.dist



class Pairwise:
    """Defines a pairwise factor, that is a function over two random variables."""
    
    def __init__(self, dist, rv0, rv0l, rv0k, rv1, rv1l, rv1k):
        """The parameters are somewhat complicated - the first is the joint distribution over
        all random variables this RV is connected to (not necesary for all distributions, but
        required when dealing with Gaussian distributions). You then have two sets of three
        parameters, the first being link 0, the second link 1 for this factor. These three
        parameters are: (rvX: The random variable object it is connected to; rvXl: The link
        of the random variable it is attached to; rvXk: The indices of the joint distribution
        to keep when sending a message down this link, such that any omitted are marginalised out)"""
        # Calculate the inverse of the k parameters - to expand incomming messages to
        # align with the joint distribution...
        rv0ik = [None] * dist.dims()
        for i, val in enumerate(rv0k):
            rv0ik[val] = i
        
        rv1ik = [None] * dist.dims()
        for i, val in enumerate(rv1k):
            rv1ik[val] = i
        
        # Record information...
        self.dist = dist
        self.link0 = (rv0, rv0l, rv0k, rv0ik)
        self.link1 = (rv1, rv1l, rv1k, rv1ik)
       
    
    def send(self, link):
        """Sends the requested message down the link, throwing an error if the
        required information is not avaliable."""
        
        # Work out which way round we are going...
        if link==0:
            source = self.link1
            dest = self.link0
        
        else: # link==1
            source = self.link0
            dest = self.link1
            
        # Fetch the message from the other side, make a duplicate so its not corrupted...
        other = source[0].msg_out[source[1]]
        if other==None:
            raise RuntimeError('Attempt to calculate message when Pairwise has not received dependent messages.')
        other = Gaussian(other)
            
        # Expand it to cover the parameters of this distribution...
        other.rejig(source[3])
        
        # Multiply by the joint...
        other *= self.dist
            
        # Marginalise down to the dimensions the destination RV cares about...
        other.rejig(dest[2])
            
        # Send message...
        dest[0].msg_in[dest[1]] = other


In [4]:
# 4. Need to be able to convert offsets to joint distributions for
# the pairwise factors - this is very problem specific...

def offset_to_joint2D(ox, oy, sd):
    mean = 0.5 * numpy.array([-ox,-oy,ox,oy])
    prec = numpy.array([[ 1.0, 0.0,-1.0, 0.0],
                        [ 0.0, 1.0, 0.0,-1.0],
                        [-1.0, 0.0, 1.0, 0.0],
                        [ 0.0,-1.0, 0.0, 1.0]])
    prec /= sd**2
    pmean = prec.dot(mean)
    
    # Convert it into a Gaussian - it's a rank 2 matrix so can't use normal constructor,
    # due to degeneracy...
    ret = Gaussian(4)
    ret.prec = prec
    ret.pmean = pmean
    return ret


In [5]:
# 5. Construct the graphical model...

# Create random variables, all have two factors...
DD = RV(2)
N1 = RV(2)
N2 = RV(2)
N3 = RV(2)

# Add unary variables - using the convention that forwards is link 0 to link 1 for RVs...
whisky_oat_zombie_mean = numpy.array([-numpy.sqrt(0.5*7**2), numpy.sqrt(0.5*7**2)])
whisky_oat_zombie_covar = numpy.array([[5**2, 0.0], [0.0, 5**2]])
whisky_oat_zombie = Unary(Gaussian(whisky_oat_zombie_mean, whisky_oat_zombie_covar), DD, 0)

step4_mean = numpy.array([-numpy.sqrt(0.5 * (4 * 0.58)**2), 30 -  numpy.sqrt(0.5 * (4 * 0.58)**2)])
step4_covar = numpy.array([[(4 * 0.04)**2, 0.0], [0.0, (4 * 0.04)**2]])
step4 = Unary(Gaussian(step4_mean, step4_covar), N3, 1)

# Now the pairwise terms...
step1 = Pairwise(offset_to_joint2D(32*0.58, 0.0, 32*0.04), DD, 1, [0,1], N1, 0, [2,3])
step2 = Pairwise(offset_to_joint2D(0.0, 45*0.58, 45*0.04), N1, 1, [0,1], N2, 0, [2,3])

edge22 = numpy.sqrt(0.5 * (22*0.58)**2)
step3 = Pairwise(offset_to_joint2D(-edge22, -edge22, 22*0.04), N2, 1, [0,1], N3, 0, [2,3])

In [6]:
# 6. Do a forward and backwards pass. In this exact case we actually
# only need a single marginal, so could just do a pass towards
# DD and stop there, but this way we can print out all of the marginals...

# Forward...
whisky_oat_zombie.send(0)
DD.send(1)
step1.send(1)
N1.send(1)
step2.send(1)
N2.send(1)
step3.send(1)
# N3.send(1) # Works, but pointless to send a message to a unary term.


# Backwards...
step4.send(0)
N3.send(0)
step3.send(0)
N2.send(0)
step2.send(0)
N1.send(0)
step1.send(0)
# DD.send(0) # As for N3 above.


In [7]:
# 7. Print out all of the marginals, where the DD marginal is the desired answer...
# (If you're wondering about all the zeroes in the covariance matrices note
# how we actually have two independent problems, one for x and one for y,
# because all distributions are isotropic)

for name, node in [('Drunk Duck', DD), ('After step 1', N1), ('After step 2', N2), ('After step 3', N3)]:
    mean, covar = node.belief().mean_covar()
    print(name + ':')
    print('  mean = ({:.2f}, {:.2f})'.format(mean[0],mean[1]))
    print('  covar = [[{:.2f}, {:.2f}],'.format(covar[0,0], covar[0,1]))
    print('           [{:.2f}, {:.2f}]]'.format(covar[1,0], covar[1,1]))
    print('')


Drunk Duck:
  mean = (-10.03, 10.11)
  covar = [[4.63, 0.00],
           [0.00, 4.63]]

After step 1:
  mean = (8.20, 10.45)
  covar = [[3.51, 0.00],
           [0.00, 3.51]]

After step 2:
  mean = (7.54, 37.22)
  covar = [[0.78, 0.00],
           [0.00, 0.78]]

After step 3:
  mean = (-1.64, 28.35)
  covar = [[0.03, 0.00],
           [0.00, 0.03]]

